In [1]:
#@title Texto de título predeterminado
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,014 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,282 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restricte

In [33]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=40a69307f8343d7e6aa8b8401719fb8e75dd47a8d6828fddf98dbaa1b9bb3c0a
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [6]:
from pyspark.sql.functions import year

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [49]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
homsal_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
homsal_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [50]:
# 2. Create a temporary view of the DataFrame.
homsal_df.createOrReplaceTempView('home_sales')


In [52]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("SELECT ROUND(AVG(price), 2), year  FROM home_sales WHERE bedrooms==4 GROUP BY year").show()

AnalysisException: ignored

In [54]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
    SELECT ROUND(AVG(price), 2), date_built
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
"""
result = spark.sql(query)
result.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           290555.07|      2016|
|           293683.19|      2012|
|           292676.79|      2017|
|           290852.27|      2014|
|           295962.27|      2013|
|           291117.47|      2011|
|            288770.3|      2015|
|           292859.62|      2010|
+--------------------+----------+



In [55]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
    SELECT ROUND(AVG(price), 2), date_built
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND sqft_living >= 2000
    GROUP BY date_built
"""
result = spark.sql(query)
result.show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           297595.75|      2016|
|           305637.43|      2012|
|           298855.99|      2017|
|           290131.71|      2014|
|           305310.69|      2013|
|           296642.44|      2011|
|            298521.3|      2015|
|           301208.84|      2010|
+--------------------+----------+



In [56]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = "SELECT  ROUND(AVG(price), 2), view  FROM home_sales WHERE price>=350000 GROUP BY view"

spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           403005.77|   7|
|           788128.21|  51|
|            404673.3|  15|
|           798684.82|  54|
|           399548.12|  11|
|           397771.65|  29|
|           750537.94|  69|
|            396964.5|  42|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            398867.6|   3|
|            397862.0|  30|
|           401419.75|  34|
|            791453.0|  59|
|           398592.71|   8|
|           402124.62|  28|
|           402022.68|  22|
|          1056336.74|  85|
|           399586.53|  16|
+--------------------+----+
only showing top 20 rows

--- 0.6779403686523438 seconds ---


In [57]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [58]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [59]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = "SELECT  ROUND(AVG(price), 2), view  FROM home_sales WHERE price>=350000 GROUP BY view"

spark.sql(query).show()



print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           403005.77|   7|
|           788128.21|  51|
|            404673.3|  15|
|           798684.82|  54|
|           399548.12|  11|
|           397771.65|  29|
|           750537.94|  69|
|            396964.5|  42|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            398867.6|   3|
|            397862.0|  30|
|           401419.75|  34|
|            791453.0|  59|
|           398592.71|   8|
|           402124.62|  28|
|           402022.68|  22|
|          1056336.74|  85|
|           399586.53|  16|
+--------------------+----+
only showing top 20 rows

--- 1.068840503692627 seconds ---


In [62]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homsal_df.write.parquet('home_parquet', mode='overwrite')




In [63]:
# 11. Read the parquet formatted data.
parquetdf = spark.read.parquet('home_parquet')


In [64]:
# 12. Create a temporary table for the parquet data.
parquetdf.createOrReplaceTempView('home_parquet')

In [65]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_parquet group by view having ROUND(AVG(price),2) >= 350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.7729926109313965 seconds ---


In [66]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [67]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
    print('home_sales is cached')
else:
    print('home_sales is no longer cached')

home_sales is no longer cached
